In [1]:

import cv2                 
import numpy as np         
import os                  
from random import shuffle 
from tqdm import tqdm      
import matplotlib
from sklearn import decomposition
IMG_SIZE=50

import pandas as pd
df = pd.read_csv ('E:/Multi_Chest/train_.csv')



In [2]:

"""
# In[2]:


def label_img_dis(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [ltn,gtn]
    #ltn means age is less than ten and gtn means age is greater than ten                           
    if (word_label == 'nor') : return np.array( [0])
    elif (word_label == 'cov') : return np.array( [1])
    elif (word_label == 'vir') : return np.array( [2])
    
"""    
    
total=112100   
import cv2 
import numpy as np 

def rotate15(img):
    try: 
        # Read image from disk. 
         
  
        # Get number of pixel horizontally and vertically. 
        (height, width) = img.shape[:2] 
  
        # Specify the size of image along with interploation methods. 
        # cv2.INTER_AREA is used for shrinking, whereas cv2.INTER_CUBIC 
        # is used for zooming. 
        res = cv2.resize(img, (int(width / 2), int(height / 2)), interpolation = cv2.INTER_CUBIC) 
  
        # Write image back to disk. 
        #cv2.imwrite('E:/fanew.png', res) 
    except IOError:
        print ('Error while reading files !!!') 
    
    try:
        # Shape of image in terms of pixels. 
        (rows, cols) = res.shape[:2] 
  
        # getRotationMatrix2D creates a matrix needed for transformation. 
        # We want matrix for rotation w.r.t center to 45 degree without scaling. 
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), 3, 1) 
        res = cv2.warpAffine(res, M, (cols, rows)) 
  
        # Write image back to disk. 
             
    except IOError:
        print ('Error while reading files !!!') 
    return res

def rotate_15(img,num):
    try: 
        # Read image from disk. 
         
  
        # Get number of pixel horizontally and vertically. 
        (height, width) = img.shape[:2] 
  
        # Specify the size of image along with interploation methods. 
        # cv2.INTER_AREA is used for shrinking, whereas cv2.INTER_CUBIC 
        # is used for zooming. 
        res = cv2.resize(img, (int(width / 2), int(height / 2)), interpolation = cv2.INTER_CUBIC) 
  
        # Write image back to disk. 
        #cv2.imwrite('E:/fanew.png', res) 
    except IOError:
        print ('Error while reading files !!!') 
    
    try:
        # Shape of image in terms of pixels. 
        (rows, cols) = res.shape[:2] 
  
        # getRotationMatrix2D creates a matrix needed for transformation. 
        # We want matrix for rotation w.r.t center to 45 degree without scaling. 
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), num, 1) 
        res = cv2.warpAffine(res, M, (cols, rows)) 
  
        # Write image back to disk. 
             
    except IOError:
        print ('Error while reading files !!!') 
    return res
    

    
from random import randint
   
def create_train_data():
    training_data = []
    i=1
    pbar = tqdm(total=total)#total is number of image in csv file for training labels
    while i<=total:
        value = randint(-26,26)
        img = cv2.imread('D:/All/{}'.format(df['Image Index'][i]),0)#Directory could be changed accordingly
        #img=rotate_15(img,value)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        vmax = max(img.max(), -img.min())


        n_components = 20

        estimator = decomposition.NMF(n_components = n_components, init = 'random', tol=5e-3)    
        W = estimator.fit_transform(img)
        H = estimator.components_

        new_img = np.dot(W,H)
        cv2.imwrite('E://1.png',new_img)
        new_img=cv2.imread('E://1.png')
        lab= cv2.cvtColor(new_img, cv2.COLOR_BGR2LAB)
        #cv2.imshow("lab",lab)

        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #cv2.imshow('CLAHE output', cl)

        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        cv2.imwrite('E://1.png',final)
        imgnmf=cv2.imread('E://1.png',0)
        labelA=[df['A1'][i],df['A2'][i],df['A3'][i],df['A4'][i]]
        labelG=[df['Male'][i],df['Fmale'][i]]
        labelD=[df['Atelectasis'][i],df['Cardiomegaly'][i],df['Consolidation'][i],df['Edema'][i],df['Effusion'][i],df['Emphsema'][i],df['Fibrosis'][i],df['No Finding'][i],df['Hernia'][i],df['Infiltration'][i],df['Mass'][i],df['Nodule'][i],df['Plural Thickning'][i],df['Pneumonia'][i],df['Pneumothorax'][i]]
        training_data.append([np.array(imgnmf),np.array(img),np.array(labelD),np.array(labelG),np.array(labelA)])
        pbar.update(1)
        i=i+1
    pbar.close()
    shuffle(training_data)
    np.save('E:/thorax_data.npy', training_data)
    return training_data








In [3]:
train=create_train_data()


  2%|█                                                                         | 1682/112100 [01:23<1:34:25, 19.49it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
  2%|█▌                                                                        | 2304/112100 [01:57<1:37:14, 18.82it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
  2%|█▋                                                                        | 2647/112100 [02:16<1:28:27, 20.62it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to im

 23%|████████████████▉                                                        | 26086/112100 [44:24<5:36:30,  4.26it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 24%|█████████████████▊                                                       | 27386/112100 [46:57<2:55:48,  8.03it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 25%|██████████████████▏                                                      | 27854/112100 [47:54<3:24:03,  6.88it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to im

 43%|██████████████████████████████▍                                        | 48120/112100 [1:25:41<3:21:55,  5.28it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 43%|██████████████████████████████▋                                        | 48521/112100 [1:27:20<2:36:45,  6.76it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 43%|██████████████████████████████▊                                        | 48738/112100 [1:27:44<3:32:58,  4.96it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to im

 66%|███████████████████████████████████████████████                        | 74365/112100 [2:23:14<1:18:20,  8.03it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 68%|████████████████████████████████████████████████▎                      | 76328/112100 [2:27:59<1:48:23,  5.50it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 68%|████████████████████████████████████████████████▍                      | 76509/112100 [2:28:27<1:11:41,  8.27it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to im

 85%|██████████████████████████████████████████████████████████████▏          | 95589/112100 [3:09:12<33:23,  8.24it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 85%|██████████████████████████████████████████████████████████████▎          | 95771/112100 [3:09:34<14:00, 19.43it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 87%|███████████████████████████████████████████████████████████████▏         | 97060/112100 [3:11:44<40:50,  6.14it/s]C:\Users\mumta\.conda\envs\tf\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to im